### Reading in the csv for cc and loyalty, setting encoding to latin-1

In [5]:
import pandas as pd
df_loyalty = pd.read_csv("loyalty_data.csv",encoding="latin-1")
df_cc = pd.read_csv("cc_data.csv",encoding="latin-1")

### 1. scatterplot comparing the two based on price to see if there is any similarity between the cards


In [6]:
import plotly.graph_objects as go
import pandas as pd

# Sort dataframes by price column
df_loyalty_sorted = df_loyalty.sort_values('price')
df_cc_sorted = df_cc.sort_values('price')

# Separate date and time for credit card data
df_cc['date'] = pd.to_datetime(df_cc['timestamp']).dt.date
df_cc['time'] = pd.to_datetime(df_cc['timestamp']).dt.time
df_cc_sorted = df_cc.sort_values('price')

# Convert timestamp to numerical format
df_loyalty_sorted['timestamp_numeric'] = pd.to_datetime(df_loyalty_sorted['timestamp'], format='%m/%d/%Y').astype('int64')
df_cc_sorted['timestamp_numeric'] = pd.to_numeric(pd.to_datetime(df_cc_sorted['timestamp']))

# Create hover text for loyalty data
hover_text_loyalty = [
    f"Location: {location}<br>Loyalty Number: {loyaltynum}<br>Price: {price}"
    for location, loyaltynum, price in zip(
        df_loyalty_sorted["location"],
        df_loyalty_sorted["loyaltynum"],
        df_loyalty_sorted["price"]
    )
]

# Create hover text for credit card data
hover_text_cc = [
    f"Location: {location}<br>Last 4 CC Number: {last4ccnum}<br>Price: {price}"
    for location, last4ccnum, price in zip(
        df_cc_sorted["location"],
        df_cc_sorted["last4ccnum"],
        df_cc_sorted["price"]
    )
]

# Create scatter trace for loyalty data
scatter_loyalty = go.Scatter(
    x=df_loyalty_sorted["price"],
    y=df_loyalty_sorted["loyaltynum"],
    mode="markers",
    name="Loyalty Data",
    hovertext=hover_text_loyalty,
    hovertemplate="%{hovertext}<extra></extra>"
)

# Create scatter trace for credit card data
scatter_cc = go.Scatter(
    x=df_cc_sorted["price"],
    y=df_cc_sorted["last4ccnum"],
    mode="markers",
    name="Credit Card Data",
    hovertext=hover_text_cc,
    hovertemplate="%{hovertext}<extra></extra>"
)

# Create layout for the plot
layout = go.Layout(
    title="Loyalty and Credit Card Data",
    xaxis=dict(title="Price"),
    yaxis=dict(title="Loyalty Number / Last 4 CC Number"),
    hovermode="closest"
)

# Create figure and add traces
fig = go.Figure(data=[scatter_loyalty, scatter_cc], layout=layout)

# Show the figure
fig.show()


 ### conclusion: there seems to be duplicates of transactions since they have the exact prices in some cases. we can conclude that the two are linked and can concatenate the two datasets and infer timestamps to the loyalty cards that are related to the cc transactions. We can concatenate the two datasets and we drop the instances wherer there is no link between the two meaning we remove duplicates and we remove instances where there is no link between loyalty and credit card

In [7]:


# splitting date and time for credit
df_cc[['date', 'time']] = df_cc['timestamp'].str.split(
    ' ', n=1, expand=True)
df_cc.drop('timestamp', axis=1, inplace=True)
df_loyalty.rename(columns={'timestamp': 'date'}, inplace=True)

# adding card type
df_cc['cardtype'] = 'credit'
df_loyalty['cardtype'] = 'loyalty'

# concatenating the dataframes
df = pd.concat([df_cc, df_loyalty], ignore_index=True)
# Convert to datetime if not already in datetime format
df['date'] = pd.to_datetime(df['date'])
df['day'] = df['date'].dt.day_name()
df['last4ccnum'] = df['last4ccnum'].astype(str)

df_sorted = df.sort_values("price")

# Iterate over the dataframe
for index, row in df_sorted.iterrows():
    if row['cardtype'] == 'loyalty':
        # Check if there is a transaction with the same price and date from a 'credit' cardtype
        matching_row = df_sorted[(df_sorted['cardtype'] == 'credit') &
                                 (df_sorted['price'] == row['price']) &
                                 (df_sorted['date'] == row['date'])].head(1)

        if not matching_row.empty:
            # Assign the time to be the same as the matching 'credit' transaction
            df_sorted.at[index, 'time'] = matching_row.iloc[0]['time']
            df_sorted.at[index, 'last4ccnum'] = matching_row.iloc[0]['last4ccnum']

#print(df_merged.head())

# Define a mask for duplicated rows based on date, time, and price
duplicated_mask = df_sorted.duplicated(subset=['date', 'time', 'price'], keep='first') | df_sorted.duplicated(subset=['date', 'time', 'price'], keep='last')

# Filter the mask further to include only rows where cardtype is 'credit'
duplicated_credit_mask = duplicated_mask & (df_sorted['cardtype'] == 'credit')

# Drop these rows
df_sorted = df_sorted[~duplicated_credit_mask]

df_sorted = df_sorted[df_sorted['loyaltynum'].notna()]
df_sorted['date'] = pd.to_datetime(df_sorted['date']).dt.date

### 2. adding missing locations to the location data based on the map

In [8]:
# Define a dictionary of location coordinates
location_coordinates = {
    "Brew've Been Served": (36.055821, 24.903190),
    "Hallowed Grounds": (36.062653, 24.885016),
    "Coffee Shack": (36.073711, 24.859376),
    "Bean There Done That": (36.081424, 24.851061),
    "Jack's Magical Beans": (36.067639, 24.874091),
    "Coffee Cameleon": (36.055057, 24.890666),
    "Katerina's Café": (36.055057, 24.899254),
    "Guy's Gyros": (36.058886, 24.898980),
    "Ouzeri Elian": (36.052814, 24.872723),
    "Gelatogalore": (36.059816, 24.862001),
    "Frydos Autosupply n' More": (36.058722, 24.906310),
    "Ahaggo Museum": (36.075297, 24.878248),
    "Albert's Fine Clothing": (36.076364, 24.857270),
    "Frank's Fuel": (36.072905, 24.838942),
    "U-Pump": (36.067748, 24.868730),
    "Roberts and Sons": (36.063394, 24.852348),
    "General Grocer": (36.060691, 24.857844),
    "Chostus Hotel": (36.069663, 24.892744),
    "Carlyle Chemical Inc.": (36.059324, 24.882788),
    "Maximum Iron and Steel": (36.063521, 24.837565),
    "Abila Scrapyard": (36.074517, 24.845935),
    "Abila airport" : (36.050285,24.828247),
    "Pilau park" :  (36.054886,24.841749),
    "Kronos Capitol": (36.052979,24.846714),
    "Kronos Mart" : (36.066542,24.847985),
    "Abila hospital" : (36.054947,24.878925),
    "Gastech" : (36.047442,24.879901),
    "someones home" :(36.063813,24.906964),
    "sannan park" : (36.066320,24.902103),
    "House2" : (36.08449,24.85915),
    "Parla park" : (36.089571,24.853747),
    "spetson park" : (36.078781,24.867915),
    "Taxisarchon park" : (36.070665,24.877297),
    "arkadio park" :(36.067288,24.890527),
    "Nationwide refinery" : (36.0558297,24.8863381),
    "suspicious place" : (24.89627, 36.006282)
}


# Add new columns "lat" and "long" based on the location coordinates
df_sorted["lat"] = df_sorted["location"].map(lambda x: location_coordinates[x][0] if x in location_coordinates else None)
df_sorted["long"] = df_sorted["location"].map(lambda x: location_coordinates[x][1] if x in location_coordinates else None)




df_location_coordinates = pd.DataFrame.from_dict(location_coordinates, orient='index', columns=['lat', 'long'])

# Reset index
df_location_coordinates.reset_index(inplace=True)

# Rename 'index' column to 'location'
df_location_coordinates.rename(columns={'index': 'location'}, inplace=True)


# Export DataFrame into a CSV file
df_location_coordinates.to_csv('location_coordinate.csv')
print(df_sorted['location'].unique())
print(df_location_coordinates.head())

["Brew've Been Served" 'Hallowed Grounds' 'Coffee Cameleon'
 "Jack's Magical Beans" 'Brewed Awakenings' 'Coffee Shack'
 'Bean There Done That' 'Katerina\x92s Café' 'Abila Zacharo' "Guy's Gyros"
 'Ouzeri Elian' 'Hippokampos' 'Gelatogalore' 'Kalami Kafenion'
 "Shoppers' Delight" "Frydos Autosupply n' More" 'Ahaggo Museum'
 "Albert's Fine Clothing" "Frank's Fuel" 'U-Pump' 'Roberts and Sons'
 "Octavio's Office Supplies" 'General Grocer' 'Kronos Mart'
 'Chostus Hotel' 'Desafio Golf Course' 'Abila Airport'
 'Nationwide Refinery' 'Carlyle Chemical Inc.'
 'Kronos Pipe and Irrigation' 'Stewart and Sons Fabrication'
 'Maximum Iron and Steel' 'Abila Scrapyard']
               location        lat       long
0   Brew've Been Served  36.055821  24.903190
1      Hallowed Grounds  36.062653  24.885016
2          Coffee Shack  36.073711  24.859376
3  Bean There Done That  36.081424  24.851061
4  Jack's Magical Beans  36.067639  24.874091


### 3. concatenate gps and car-assignment to work with one dataset for gps. 

In [9]:
import pandas as pd

# Read the GPS data from csv
df_gps = pd.read_csv("gps1.csv")
df_car = pd.read_csv("car-assignments.csv")
df_car['FullName'] = df_car['FirstName'] + ' ' + df_car['LastName']

# Merge df_gps and df_car based on 'id' in df_gps and 'CarID' in df_car
df_merged = pd.merge(df_gps, df_car, left_on='id', right_on='CarID', how='left')

df_merged = pd.merge(df_gps, df_car, left_on='id', right_on='CarID', how='inner')
# Combine 'FirstName' and 'LastName' into a new column 'FullName'
# Drop 'FirstName' and 'LastName' columns
df_merged = df_merged.drop(['FirstName', 'LastName'], axis=1)

print(df_merged.head())

df_merged.to_csv('gps.csv', index=False)



             Timestamp  id        lat       long  CarID CurrentEmploymentType   
0  01/06/2014 06:28:01  35  36.076225  24.874689   35.0             Executive  \
1  01/06/2014 06:28:01  35  36.076220  24.874596   35.0             Executive   
2  01/06/2014 06:28:03  35  36.076211  24.874443   35.0             Executive   
3  01/06/2014 06:28:05  35  36.076217  24.874253   35.0             Executive   
4  01/06/2014 06:28:06  35  36.076214  24.874167   35.0             Executive   

         CurrentEmploymentTitle           FullName  
0  Environmental Safety Advisor  Willem Vasco-Pais  
1  Environmental Safety Advisor  Willem Vasco-Pais  
2  Environmental Safety Advisor  Willem Vasco-Pais  
3  Environmental Safety Advisor  Willem Vasco-Pais  
4  Environmental Safety Advisor  Willem Vasco-Pais  


### cleaning: we dop na values for time and where lat or long is not defined. 

In [10]:
df_sorted = df_sorted.dropna(subset=['time'])

# Check for NaN values in 'lat' and 'long' columns for df_sorted
print(df_sorted[df_sorted['lat'].isna() | df_sorted['long'].isna()])

# Check for NaN values in 'lat' and 'long' columns for df_merged
print(df_merged[df_merged['lat'].isna() | df_merged['long'].isna()])

# Drop rows with NaN lat/long values from df_sorted
df_sorted = df_sorted.dropna(subset=['lat', 'long'])

# Drop rows with NaN lat/long values from df_merged
df_merged = df_merged.dropna(subset=['lat', 'long'])

                          location    price last4ccnum        date   time   
2533             Brewed Awakenings     3.62     8332.0  2014-01-15  12:00  \
1797             Brewed Awakenings     4.35     8332.0  2014-01-06  12:00   
1782             Brewed Awakenings     5.72     7688.0  2014-01-06  12:00   
2726             Brewed Awakenings     6.00     8332.0  2014-01-17  12:00   
2171             Brewed Awakenings     6.17     2540.0  2014-01-10  12:00   
...                            ...      ...        ...         ...    ...   
2111                 Abila Airport  4538.52     8642.0  2014-01-09  15:50   
1822                 Abila Airport  4540.08     9220.0  2014-01-06  08:23   
2673  Stewart and Sons Fabrication  4543.31     9735.0  2014-01-16  11:45   
2671           Nationwide Refinery  4742.67     7792.0  2014-01-16  11:25   
2138                 Abila Airport  4792.50     2276.0  2014-01-09  13:18   

     cardtype loyaltynum        day  lat  long  
2533  loyalty      L2070  

### 4. majority vote algorithm to infer cc to owner

In [11]:
from datetime import timedelta
from geopy.distance import geodesic
from collections import defaultdict
import pandas as pd

# A nested dictionary to keep track of scores for each last4ccnum and FullName pair
proximity_scores = defaultdict(lambda: defaultdict(int))

def update_proximity_scores(transaction, df_merged):
    transaction_datetime = pd.to_datetime(f"{transaction['date']} {transaction['time']}")

    mask = (df_merged['Timestamp'].dt.date == transaction_datetime.date()) & \
           (df_merged['Timestamp'] >= transaction_datetime - timedelta(minutes=30)) & \
           (df_merged['Timestamp'] <= transaction_datetime + timedelta(minutes=30))
    
    candidates = df_merged[mask]

    if candidates.empty:
        return

    min_distance = float('inf')
    closest_person = None

    for _, row in candidates.iterrows():
        distance = geodesic((transaction['lat'], transaction['long']), (row['lat'], row['long'])).meters
        if distance < min_distance:
            min_distance = distance
            closest_person = row['FullName']

    if closest_person:
        proximity_scores[transaction['last4ccnum']][closest_person] += 1

df_merged['Timestamp'] = pd.to_datetime(df_merged['Timestamp'], errors='coerce')

# Update the proximity_scores dictionary
df_sorted.apply(lambda x: update_proximity_scores(x, df_merged), axis=1)

# Majority vote and assignment
assigned_cards = {}
assigned_people = set()

for card_num, scores in proximity_scores.items():
    # Find the person with the highest score for this card that hasn't been assigned yet
    person = max(scores, key=scores.get, default=None)
    if person and person not in assigned_people:
        assigned_cards[card_num] = person
        assigned_people.add(person)

# Add the inferred person to df_sorted based on majority vote
df_sorted['FullName'] = df_sorted['last4ccnum'].map(assigned_cards)

df_merged = df_merged.dropna(subset=['Timestamp'])

df_sorted.to_csv('cards.csv', index=False)

In [12]:
import pandas as pd
import plotly.express as px

def getHeatmap(df):
    

    return fig

df = pd.read_csv("cards.csv")

df_count = df.groupby('location').size().reset_index(name='count')
df_sorted = df_count.sort_values(by='count', ascending=False)

fig = px.density_heatmap(
    df,
    x="date", y="location",
    color_continuous_scale='YlOrRd',
    nbinsx=len(df['date'].unique()),
    nbinsy=len(df['location'].unique()),
    category_orders={"location": df_sorted['location']})  # Set category order based on count

# Reorder the locations based on the accumulated count
fig.update_yaxes(categoryarray=df_sorted['location'])

fig.show()

print(5)